# Langchain Components Demo
This demo can be downloaded from:

The full documentation on Langchain can be found at:
https://python.langchain.com/v0.1/docs/get_started/quickstart/

## Getting Started
Install the packages with pip.
1. Download and install [Anaconda Navigator](https://www.anaconda.com/download/)
2. Launch Jupyter Notebook from Anaconda Navigator
3. Go to "New" -> "Terminal"
4. Run the following command in the terminal
5. Open this `demo.ipynb` in Jupyter Notebook

```bash
pip install -U langchain==0.2.0
pip install -U langchain-core==0.2.0
pip install -U langchain-community==0.2.0
pip install -U langchain-openai==0.1.7
pip install -U langchain-experimental==0.0.59
pip install -U wikipedia
pip install -U yfinance
pip install -U tabulate
```

In [1]:
# Setting API Key and environment variables
import os

os.environ["AZURE_OPENAI_API_KEY"] = "685158aec57747dfad586110be180657"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://anv2dev-azureopenai-auseast.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "anv2dev-gpt35turbo1106-auseast"

## 1. Prompt Template
Prompt templates are **predefined recipes for generating prompts** for language models such as OpenAI's GPT model.

A template may include:
- instructions, 
- few-shot examples, and 
- specific context and questions appropriate for a given task.

Documentation: [Prompt Templates](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/)



### 1.1 PromptTemplate
Use PromptTemplate to create a template for a string prompt.

In [2]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

### 1.2 PromptTemplate with `input variables` and `partial variables`

Like other methods, it can make sense to "partial" a prompt template - e.g. `pass in a subset of the required values`, as to create a new prompt template which expects only the remaining subset of values.



In [3]:
prompt_template = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    input_variables=["adjective", "content"],
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [4]:
prompt_template = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    input_variables=["content"],
    partial_variables={"adjective": "funny"},
)
prompt_template.format(adjective="hilarious", content="chickens")

'Tell me a hilarious joke about chickens.'

### 1.3 ChatPromptTemplate
ChatPromptTemplate is a template to `chat models` and it contains a list of `chat messages` that can be used to interact with the model.
Each chat message is associated with content, and an additional parameter called `role`. 

For example, in the `OpenAI Chat Completions API`, a chat message can be associated with an `AI assistant`, a `human` or a `system` role.

Let's create a chat template for `OpenAI chat model`.

In [5]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

Let's invoke the chat model with the generated messages.

In [6]:
from langchain_openai import AzureChatOpenAI

# Initialize the OpenAI Chat model
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

model.invoke(messages)

AIMessage(content='My name is Bob. How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 50, 'total_tokens': 62}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-bc5b06fc-e655-40ac-82de-acb8d6168b55-0')

### 1.4 MessagesPlaceholder
`MessagesPlaceholder` gives you full control of what messages to be rendered during formatting. 

This can be useful when you are uncertain of:
- what **role** you should be using for your message prompt templates or ;
- when you wish to insert a **list of messages** during formatting

Let's create a simple chat prompt using `MessagesPlaceholder`:

In [7]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="some_conversation"), human_message_template]
)
chat_prompt

ChatPromptTemplate(input_variables=['some_conversation', 'word_count'], input_types={'some_conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='some_conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='Summarize our conversation so far in {word_count} words.'))])

Let's format the `chat_prompt` with some conversation messages.

In [8]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(
    content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

# Format the chat prompt by passing the conversation messages
messages = chat_prompt.format_prompt(
    some_conversation=[human_message, ai_message], word_count="10"
).to_messages()

messages


[HumanMessage(content='What is the best way to learn programming?'),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience'),
 HumanMessage(content='Summarize our conversation so far in 10 words.')]

Let's invoke the Open AI chat model with the generated `messages`.

In [9]:
model.invoke(messages)

AIMessage(content='Learning programming: choose language, start with basics, practice consistently.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 99, 'total_tokens': 112}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-5278d68d-2d9d-4a2b-9603-9a8c6cf88f4a-0')

## 2. Output Parsers
Output parsers are responsible for taking the output of an LLM and **transforming it to a more suitable format** e.g `.json`, `.csv`. 

This is very useful when you are using LLMs to generate any form of structured data.

Documentation: [Output Parsers](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/)

### 2.1 Simple JsonOutputParser
This is a simple output parser that converts the output of an LLM to a JSON object.

In [10]:
from langchain_core.output_parsers import JsonOutputParser

# Simple Json Output Parser
parser = JsonOutputParser()

# Get format instructions for the prompt template.
format_instructions = parser.get_format_instructions()

# Define a prompt template with the parser's format instructions.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)
prompt


PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'Return a JSON object.'}, template='Answer the user query.\n{format_instructions}\n{query}\n')

Let's invoke the model with the `prompt` and `user_query`.

In [11]:
import json

# Initialize the OpenAI Chat model
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0.0
)

# A query intended to prompt a language model to populate the data structure.
user_query = "What car has a very good horse power?"

# Invoke the model with the prompt
# model.invoke(prompt.format(query=user_query))

content = model.invoke(prompt.format(query=user_query)).content

json_obj = json.loads(content)
json_obj


{'car': 'Tesla Model S', 'horsepower': '762'}

### 2.2 JsonOutputParser with Pydantic
This output parser allows users to specify an **arbitrary JSON schema** and query LLMs for outputs that **conform to that schema**.


In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Define your desired data structure for JSON.
class Car(BaseModel):
    brand: str = Field(description="the brand of the car in string")
    model: str = Field(description="the model of the car in string")
    hp: int = Field(description="the horse power of the car in integer")


# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Car)

# Get format instructions for the prompt template.
format_instructions = parser.get_format_instructions()

# Define a prompt template with the parser's format instructions.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)
prompt

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"brand": {"title": "Brand", "description": "the brand of the car in string", "type": "string"}, "model": {"title": "Model", "description": "the model of the car in string", "type": "string"}, "hp": {"title": "Hp", "description": "the horse power of the car in integer", "type": "integer"}}, "required": ["brand", "model", "hp"]}\n```'}, template='Answer the user query.\n{format_instructions}\n{query}\n')

Let's invoke the model with the `prompt` and `user_query`.

In [13]:
# A query intended to prompt a language model to populate the data structure.
user_query = "What car has a very good horse power?"

# Invoke the model with the prompt
# model.invoke(prompt.format(query=user_query))

content = model.invoke(
    prompt.format(query=user_query)
).content

json_obj = json.loads(content)
json_obj

{'brand': 'Tesla', 'model': 'Model S', 'hp': 503}

### 2.3 CSVOutputParser (Exercise)
Can you create a prompt template that output a `CSV` list of 5 `{subject}`?

E.g. List 5 `car brands`; List 5 `programming languages`;

Reference: [CommaSeparatedListOutputParser](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/csv/)


In [14]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

# Get format instructions for the prompt template.
format_instructions = parser.get_format_instructions()

# Define a prompt template with the parser's format instructions.
prompt = PromptTemplate(
    template="List 5 {subject}\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

subject = "car brands"

content = model.invoke(
    prompt.format(subject=subject)
).content

content

'Toyota, Ford, Honda, Chevrolet, BMW'

## 3. Chaining with LLMChain & LangChain Expression Language (LCEL)

A **Chain** refers to a sequence of steps or operations that are executed in a specific order to accomplish a task using LLMs and other computational components. 

Chains are a core concept in LangChain, allowing developers to build complex workflows that involve multiple interactions with:
- prompts,
- models,
- arbitrary functions,
- or even other chains

The primary supported way to do this is with `LCEL`.

Documentation: [Chaining](https://python.langchain.com/v0.1/docs/modules/chains/)



### 3.1 Simple Chaining with LLMChain


In [15]:
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("List 5 {subject}.")

parser = StrOutputParser()

chain = LLMChain(
    llm=model, 
    prompt=prompt,
    output_parser=parser,
)

chain.invoke({"subject": subject})

/Users/xeroxis/Documents/GitHub/Langchain-Demo/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'subject': 'car brands',
 'text': '1. Toyota\n2. Ford\n3. BMW\n4. Honda\n5. Mercedes-Benz'}

### 3.1 Simple Chaining with LCEL
We can use LangChain runnables to chain together prompts, models, and parsers.

LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together.

LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.

Documentation: [LCEL](https://python.langchain.com/v0.1/docs/expression_language/)

In [16]:
# Chaining different components using LCEL pipe operator
chain = prompt | model | parser

chain.invoke({"subject":subject})

'1. Toyota\n2. Ford\n3. BMW\n4. Honda\n5. Mercedes-Benz'

### 3.2 Combined Chaining with LCEL (Sequential)
We can even combine this chain with more runnables to create another chain.

In [17]:
# Define a second prompt template
analysis_prompt = ChatPromptTemplate.from_template("How do you classify these brands in terms of quality and pricing? {brands}")

combined_chain = {"brands": chain} | analysis_prompt | model | StrOutputParser()

combined_chain.invoke({"subject": subject})


'1. Toyota - Mid-range quality and mid-range pricing\n2. Ford - Mid-range quality and mid-range pricing\n3. BMW - High-quality and high pricing\n4. Honda - Mid-range quality and mid-range pricing\n5. Mercedes-Benz - High-quality and high pricing'

### 3.3 Parrallel Chaining with LCEL
RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

In [18]:
from langchain_core.runnables import RunnableParallel

joke_prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
poem_prompt = ChatPromptTemplate.from_template("write a 2-line poem about {topic}")

joke_chain = joke_prompt | model | StrOutputParser()
poem_chain = poem_prompt | model | StrOutputParser()

map_chain = RunnableParallel(
    joke=joke_chain, 
    poem=poem_chain
)
map_chain.invoke({"topic": "cat"})


{'joke': 'Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!',
 'poem': 'Whiskers soft and eyes so bright,\nPurring softly through the night.'}

### 3.4 Using custom functions within Chain
You can also use arbitrary functions in the pipeline.

In [19]:
from operator import itemgetter

from langchain_core.runnables import RunnableLambda

def get_length(text):
    return len(text)

def get_multiple_length(_dict):
    return len(_dict["text1"]) * len(_dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(get_length),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}| RunnableLambda(get_multiple_length),
    }
    | prompt
    | model
    | StrOutputParser() 
)

chain.invoke({"foo": "hi", "bar": "world"})

'2 + 10 equals 12.'

## 4. Memory
Most LLM applications have a conversational interface. 

An essential component of a conversation is being able to refer to information introduced earlier in the conversation. 

At bare minimum, a conversational system should be able to access some window of past messages directly. 

Documentation: [Memory](https://python.langchain.com/v0.1/docs/modules/memory/)

### 4.1 In-Memory with LCEL
The `RunnableWithMessageHistory` lets us add message history to certain types of chains. It wraps another `Runnable` and manages the chat message history for it.

Below we show a simple example in which the chat history lives in memory, in this case via a global Python dict.

Documentation: [Memory LCEL](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/)

In [20]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        (
            "human", 
            "{input}"
        ),
    ]
)

runnable = prompt | model

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        # Create a new chat message history
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    # the key to be treated as the latest input message
    input_messages_key="input",
    # the key to add historical messages to
    history_messages_key="history",
)

In [22]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)


AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 33, 'total_tokens': 59}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-573739ef-8f1e-4063-bb84-4cd42a998b52-0')

Recalling the last message

In [23]:
# Remembers
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='Cosine is a function that relates the angle of a right triangle to the ratio of its sides.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 69, 'total_tokens': 89}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-b4614317-6a0e-45db-bef2-59ffa48e5552-0')

What if there isn't a message history to refer to?

In [24]:
# New session_id --> does not remember.
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "def234"}},
)

AIMessage(content="I can help with math problems. Just ask me a question and I'll do my best to assist you.", response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 30, 'total_tokens': 52}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-9e7a8838-478c-4098-aef8-f16a721cf2e3-0')

## 5. Tools and Toolkits
Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

- The name of the tool
- A description of what the tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the result of a tool should be returned directly to the user

It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.

Documentation: [Toolkits](https://python.langchain.com/v0.1/docs/modules/tools/)

### 5.1 Default Tools (Wikipedia Query Tool)
The Wikipedia Query tool allows you to query Wikipedia for information.


In [25]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

Important information for LLM to recoginise and make use of the tool.

In [26]:
tool.name

'wikipedia'

In [27]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [28]:
tool.args

{'query': {'title': 'Query', 'type': 'string'}}

Let's run the tool with a query!

In [29]:
tool.run({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

### 5.2 Yahoo Finance News Tool (Exercise)
The Yahoo Finance News tool allows you to query Yahoo Finance for news articles.

Hint: use `tool.name`, `tool.description`, and `tool.args` to understand the tool.

In [39]:
from langchain_community.tools import YahooFinanceNewsTool

tool = YahooFinanceNewsTool()

tool.run({"query": "NFLX"})


'5 Must-Buy High-Flying U.S. Giants on Favorable Economic Data\nWe have narrowed our search to five U.S. giants that have provided more than 20% returns year to date with more upside left. These are: NVDA, NFLX, PGR, GOOGL, WFC.'

### 5.3 Toolkits
Toolkits are collections of tools that are designed to be used together for specific tasks. They have convenient loading methods. 

For a complete list of available ready-made toolkits, visit [Integrations](https://python.langchain.com/v0.1/docs/integrations/toolkits/).

We will demonstrate this using `Agents`.

## 6. Agents
The core idea of agents is to use a language model to choose a sequence of actions to take. 

In chains, a sequence of actions is **hardcoded** (in code). 

In agents, a language model is used as a **reasoning engine** to determine which actions to take and in which order.

Documentation: [Agents](https://python.langchain.com/v0.2/docs/integrations/toolkits/)

### 6.1 Agent with CSV Tooklits
This notebook shows how to use `agents` to interact with data in `CSV format`. 

It is mostly optimized for question answering.

In [43]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

# A zero shot agent that does a reasoning step before acting.
agent = create_csv_agent(
    model,
    "titanic.csv",
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.invoke("What is the average age of the passengers?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the average of the 'Age' column in the dataframe.
Action: python_repl_ast
Action Input: df['Age'].mean()29.69911764705882The average age of the passengers is 29.7 years.
Final Answer: 29.7 years

> Finished chain.


{'input': 'What is the average age of the passengers?', 'output': '29.7 years'}

Alternative `agent_type`:

In [44]:
# An agent optimized for using open AI functions.
agent = create_csv_agent(
    model,
    "titanic.csv",
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)
agent.invoke("What is the average age of the passengers?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Age'].mean()"}`


29.69911764705882The average age of the passengers is approximately 29.7 years.

> Finished chain.


{'input': 'What is the average age of the passengers?',
 'output': 'The average age of the passengers is approximately 29.7 years.'}

In [45]:
agent.run("how many people have more than 3 siblings")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['SibSp'] > 3]['PassengerId'].count()"}`


30There are 30 people who have more than 3 siblings.

> Finished chain.


'There are 30 people who have more than 3 siblings.'

### 6.2 Agent with Python Toolkit
We can design the agent to write and execute Python code to answer a question.

In [48]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_experimental.tools import PythonREPLTool


tools = [PythonREPLTool()]
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
agent = create_openai_functions_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is the 10th fibonacci number?"})




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci(10)`


The 10th Fibonacci number is 55.

> Finished chain.


{'input': 'What is the 10th fibonacci number?',
 'output': 'The 10th Fibonacci number is 55.'}

### 7. Huggingingface Models
The [Hugging Face](https://huggingface.co/docs/hub/index) Hub is a platform with over 120k models, 20k datasets, and 50k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.